## 📘 AIS 데이터 기반 선박 경로 필터링 및 EDA 절차
### 1. 개요

본 분석은 AIS 데이터를 활용하여 선박의 출발지-도착지 간 경로를 필터링하고, 해당 경로 데이터를 기반으로 향후 딥러닝 모델 학습을 위한 기초 자료를 수집하기 위한 탐색적 데이터 분석(EDA)을 수행하였다.

출발지와 도착지는 대한민국 인천항과 제주항으로 설정하였으며, 대용량 데이터 처리 효율을 고려하여 구체적인 공간 필터링과 샘플링 전략을 병행하였다.

### 2. 출발지 및 도착지 지역 설정

출발지와 도착지는 위도와 경도 범위로 정의된 네모난 구역(Box Region)으로 설정하여 해당 지역 내에 진입하는 선박을 식별하였다.

🗺️ 출발지 (인천항 근처) : 위도 범위: 37.3 ~ 37.6, 경도 범위: 126.4 ~ 126.8

🗺️ 도착지 (제주항 근처) : 위도 범위: 33.3 ~ 33.7, 경도 범위: 126.3 ~ 126.7

### 3. 데이터 필터링 절차
   
**<1> 출발지 통과 MMSI 식별**

PostgreSQL에서 출발지 범위 내로 최초 진입한 선박(MMSI)을 추출하였다.

MIN(timestamp)를 기준으로 출항 시각을 식별

TABLESAMPLE, WHERE 조건, 테이블 파티션 및 인덱스 등을 활용해 처리 속도 최적화


**<2> 도착지 도달 MMSI 필터링**

출발지에서 추출된 MMSI 중 출항 이후 일정 시간(한 달) 이내에 도착지 범위에 도달한 MMSI를 식별하여 유효 경로만 선별하였다.

**<3> 선박 경로 전체 추출**

출발 시각부터 도착 시각 사이에 해당 선박의 위도, 경도, 속도(SOG), 방위(COG), Heading 등의 정보를 포함한 경로 데이터를 추출하였다.


### 4. 이상치 제거 및 경로 정제

선박 경로 데이터에 포함된 위도, 경도 값 중 다음과 같은 조건을 통해 이상치를 제거하였다:

**한반도 주위 해역 위도(30 ~ 40), 경도(122 ~ 132)를 벗어나는 데이터 제거**

이를 통해 보다 매끄럽고 신뢰성 있는 이동 경로를 확보하였다.

### 5. 시각화 및 확인

필터링된 MMSI별 경로를 지도(Folium 등)를 활용해 시각화하였다.

이를 통해 실제로 선박이 출발지(인천항)에서 출발해 도착지(제주항)로 향하는지를 직관적으로 검증하였다.

### 6. CSV 파일 저장 및 재활용

정제된 경로 데이터는 이후 모델 학습, 분석, 시각화를 위한 재사용이 가능하도록 CSV 파일로 저장하였다.

이를 통해 반복적인 데이터 추출 없이도 빠른 실험이 가능하도록 하였다.

### 7. 요약

- 출발지: 	인천항 (37.3 ~ 37.6 N, 126.4 ~ 126.8 E)
- 도착지:	제주항 (33.3 ~ 33.7 N, 126.3 ~ 126.7 E)
- 분석 대상:	출발 → 도착 경로를 가진 선박(MMSI 기준)
- 주요 처리 과정:	위치 필터링 → 경로 추출 → 이상치 제거 → 시각화 → CSV 저장

In [1]:
import psycopg2
import folium
import pandas as pd
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import os

# PostgreSQL 연결 정보
DB_CONFIG = {
    "dbname": "ais_data",
    "user": "postgres",
    "password": "ky76018500",
    "host": "localhost",
    "port": "5432"
}

# 출발지 & 도착지 설정
# 인천항
departure_bounds = {"lat_min": 37.40, "lat_max": 37.50, "lon_min": 126.50, "lon_max": 126.60}
# 제주항
arrival_bounds = {"lat_min": 33.47, "lat_max": 33.57, "lon_min": 126.47, "lon_max": 126.57}

# PostgreSQL에서 데이터 가져오기
def fetch_ais_data():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    # 1️⃣ 출발지에서 MMSI 찾기
    cur.execute(f"""
        SELECT DISTINCT mmsi, MIN(timestamp) AS departure_time
        FROM ais_2020_02 TABLESAMPLE SYSTEM(5)  -- 전체 데이터 중 5%만 랜덤 샘플링
        WHERE latitude BETWEEN {departure_bounds['lat_min']} AND {departure_bounds['lat_max']}
        AND longitude BETWEEN {departure_bounds['lon_min']} AND {departure_bounds['lon_max']}
        GROUP BY mmsi;
    """)
    print(f"1단계 완료!")
    departure_mmsi = dict(cur.fetchall())

    # 2️⃣ 도착지에서 동일한 MMSI 찾기
    cur.execute(f"""
        SELECT DISTINCT mmsi, MIN(timestamp) AS arrival_time
        FROM ais_2020_02 TABLESAMPLE SYSTEM(5)  -- 전체 데이터 중 5%만 랜덤 샘플링
        WHERE latitude BETWEEN {arrival_bounds['lat_min']} AND {arrival_bounds['lat_max']}
        AND longitude BETWEEN {arrival_bounds['lon_min']} AND {arrival_bounds['lon_max']}
        AND mmsi IN ({",".join(f"'{mmsi}'" for mmsi in departure_mmsi.keys())})
        GROUP BY mmsi;
    """)
    print("2단계 완료!")
    arrival_mmsi = dict(cur.fetchall())

    # 3️⃣ MMSI별 이동 경로 가져오기
    results = []
    for mmsi, departure_time in departure_mmsi.items():
        if mmsi in arrival_mmsi:
            cur.execute(f"""
                SELECT mmsi, timestamp, latitude, longitude, sog, cog, heading
                FROM ais_2020_02 
                WHERE mmsi = '{mmsi}'
                AND timestamp BETWEEN TIMESTAMP '{departure_time}' AND TIMESTAMP '{arrival_mmsi[mmsi]}'
                ORDER BY timestamp ASC;
            """)
            print(f"3단계 완료! with {mmsi}")
            results.extend(cur.fetchall())

    # DataFrame 변환
    df = pd.DataFrame(results, columns=["mmsi", "timestamp", "latitude", "longitude", "sog", "cog", "heading"])

    # 연결 종료
    cur.close()
    conn.close()

    return df

In [2]:
# CSV 파일 경로
CSV_FILE = "ship_routes.csv"

# 데이터 가져오기 함수 (파일 존재 여부 확인)
def get_ais_data():
    if os.path.exists(CSV_FILE):
        print("📂 기존 CSV 파일을 로드합니다...")
        return pd.read_csv(CSV_FILE, parse_dates=["timestamp"])
    
    print("⏳ 데이터베이스에서 데이터를 가져오는 중...")
    df = fetch_ais_data()
    
    # 위도/경도 범위: 한반도 주변 해역 기준 (대략적 설정)
    LAT_MIN, LAT_MAX = 30, 40
    LON_MIN, LON_MAX = 122, 132

    # 위도/경도 이상치 제거
    df_clean = df[(df['latitude'].between(LAT_MIN, LAT_MAX)) & 
                  (df['longitude'].between(LON_MIN, LON_MAX))].copy()

    print(f"✅ 이상치 제거 완료: {len(df)} → {len(df_clean)} rows")
    
    if not df_clean.empty:
        df_clean.to_csv(CSV_FILE, index=False)
        print(f"✅ 데이터가 '{CSV_FILE}'로 저장되었습니다.")

    return df_clean

# 데이터 가져오기
ais_df = get_ais_data()

# 지도 생성 (한반도 중심)
m = folium.Map(location=[36.5, 127.75], zoom_start=6)

# 출발지 & 도착지 네모난 박스로 표시
folium.Rectangle(
    bounds=[[departure_bounds["lat_min"], departure_bounds["lon_min"]],
            [departure_bounds["lat_max"], departure_bounds["lon_max"]]],
    color="blue", fill=True, fill_opacity=0.3, popup="Departure Zone"
).add_to(m)

folium.Rectangle(
    bounds=[[arrival_bounds["lat_min"], arrival_bounds["lon_min"]],
            [arrival_bounds["lat_max"], arrival_bounds["lon_max"]]],
    color="red", fill=True, fill_opacity=0.3, popup="Arrival Zone"
).add_to(m)

# MMSI별 이동 경로 추가
mmsi_list = ais_df["mmsi"].unique()
print(f"MMSI로 식별되는 선박 총 개수 : {len(mmsi_list)}")
colors = plt.cm.get_cmap("tab10", len(mmsi_list))  # MMSI별 색상 지정

for i, mmsi in enumerate(mmsi_list):
    ship_data = ais_df[ais_df["mmsi"] == mmsi].sort_values(by="timestamp")
    route = list(zip(ship_data["latitude"], ship_data["longitude"]))
    folium.PolyLine(route, color=f"#{int(colors(i)[0]*255):02x}{int(colors(i)[1]*255):02x}{int(colors(i)[2]*255):02x}",
                    weight=2, opacity=0.7, popup=f"MMSI: {mmsi}").add_to(m)

# 지도 저장 및 표시
m.save("ship_routes.html")
print("✅ 경로 시각화가 완료되었습니다.")

⏳ 데이터베이스에서 데이터를 가져오는 중...
1단계 완료!
2단계 완료!
3단계 완료! with QQNQcWMDznFEjccPBIYSVw==
3단계 완료! with Uw7R0tKxRT/1JjRenDSSOQ==
✅ 이상치 제거 완료: 169399 → 148437 rows
✅ 데이터가 'ship_routes.csv'로 저장되었습니다.
MMSI로 식별되는 선박 총 개수 : 2


C:\Users\kyb87\AppData\Local\Temp\ipykernel_4076\3218653516.py:51: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap("tab10", len(mmsi_list))  # MMSI별 색상 지정


✅ 경로 시각화가 완료되었습니다.
